In [1]:
from importlib.metadata import version

pkgs = ["numpy", "tiktoken", "torch", "tensorflow"] # OpenAI의 사전 훈련된 가중치를 위해

for p in pkgs:
  print(f"{p} 버전: {version(p)}")

numpy 버전: 2.0.2
tiktoken 버전: 0.12.0
torch 버전: 2.8.0+cu126
tensorflow 버전: 2.19.0


# 연습문제 5.1: 온도 스케일링이 적용된 소프트맥스 점수 및 샘플링 확률

In [3]:
import torch

vocab = {
    "closer": 0,
    "every": 1,
    "effort": 2,
    "forward": 3,
    "inches": 4,
    "moves": 5,
    "pizza": 6,
    "toward": 7,
    "you": 8,
}

inverse_vocab = {v: k for k, v in vocab.items()}

# 입력이 "every effort moves you"이고
# LLM이 다음 토큰을 위해 아래와 같은 로짓을 반환했다 가정
next_token_logits = torch.tensor(
    [4.51, 0.89, -1.90, 6.75, 1.63, -1.62, -1.89, 6.28, 1.79]
)

def print_sampled_tokens(probas):
  torch.manual_seed(123) # 재현가능성을 위한 랜덤 시드
  sample = [torch.multinomial(probas, num_samples=1).item() for i in range(1_000)]
  sampled_ids = torch.bincount(torch.tensor(sample), minlength=len(probas))
  for i, freq in enumerate(sampled_ids):
    print(f"{freq} x {inverse_vocab[i]}")

def softmax_with_temperature(logits, temperature):
  scaled_logits = logits / temperature
  return torch.softmax(scaled_logits, dim=0)

temperatures = [1, 0.1, 5] # 원본, 높은, 그리고 낮은 온도
scaled_probas = [softmax_with_temperature(next_token_logits, T) for T in temperatures]

In [4]:
for i, probas in enumerate(scaled_probas):
  print("\n\n온도:", temperatures[i])
  print_sampled_tokens(probas)



온도: 1
73 x closer
0 x every
0 x effort
582 x forward
2 x inches
0 x moves
0 x pizza
343 x toward
0 x you


온도: 0.1
0 x closer
0 x every
0 x effort
985 x forward
0 x inches
0 x moves
0 x pizza
15 x toward
0 x you


온도: 5
165 x closer
75 x every
42 x effort
239 x forward
71 x inches
46 x moves
32 x pizza
227 x toward
103 x you


In [5]:
temp5_idx = 2
pizza_idx = 6

scaled_probas[temp5_idx][pizza_idx]

tensor(0.0430)

# 연습문제 5.2: 다양한 온도 및 top-k 설정

## 낮은 top-k 및 온도
  - 교육 콘텐츠, 기술 문서 또는 질문 응답, 데이터 분석, 코드 생성 등을 작성할 때 바람직한 덜 무작위적인 결과를 생성한다.

## 높은 top-k 및 온도
  - 브레인스토밍 작업, 창의적인 글쓰기 등에 더 바람직한 더 다양하고 무작위적인 출력을 생성한다.

# 연습문제 5.3: 디코딩 함수의 결정론적인 동작

In [6]:
!wget https://bit.ly/3HlFmc8 -O previous_chapters.py

--2025-11-15 07:20:36--  https://bit.ly/3HlFmc8
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/rickiepark/llm-from-scratch/refs/heads/main/ch05/01_main-chapter-code/previous_chapters.py [following]
--2025-11-15 07:20:36--  https://raw.githubusercontent.com/rickiepark/llm-from-scratch/refs/heads/main/ch05/01_main-chapter-code/previous_chapters.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9905 (9.7K) [text/plain]
Saving to: ‘previous_chapters.py’

previous_chapters.p 100%[===================>]   9.67K  --.-KB/s    in 0s      

2025-11-15 07:20:36 (42.5 MB/s) - ‘previous_chapter

In [7]:
!gdown 1Lze7x_4Bd3Sd22sZrCG7cxKOE2wcMKrh

Downloading...
From (original): https://drive.google.com/uc?id=1Lze7x_4Bd3Sd22sZrCG7cxKOE2wcMKrh
From (redirected): https://drive.google.com/uc?id=1Lze7x_4Bd3Sd22sZrCG7cxKOE2wcMKrh&confirm=t&uuid=964763a4-952f-4bc1-9929-370787b32b74
To: /content/model.pth
100% 653M/653M [00:11<00:00, 55.5MB/s]


In [8]:
import tiktoken
import torch
from previous_chapters import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257, # 어휘 사전 크기
    "context_length": 256, # 짧은 문맥 길이 (원본 길이: 1024)
    "emb_dim": 768, # 임베딩 차원
    "n_heads": 12, # 어텐션 헤드 개수
    "n_layers": 12, # 층 개수
    "drop_rate": 0.1, # 드롭아웃 비율
    "qkv_bias": False, # 쿼리-키-값 생성시 편향 사용 여부
}

torch.manual_seed(123)

tokenizer = tiktoken.get_encoding("gpt2")
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(torch.load("model.pth", weights_only=True))
model.eval();

In [10]:
!wget https://bit.ly/3FzTX37 -O gpt_generate.py

--2025-11-15 07:24:27--  https://bit.ly/3FzTX37
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/rickiepark/llm-from-scratch/refs/heads/main/ch05/01_main-chapter-code/gpt_generate.py [following]
--2025-11-15 07:24:27--  https://raw.githubusercontent.com/rickiepark/llm-from-scratch/refs/heads/main/ch05/01_main-chapter-code/gpt_generate.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11152 (11K) [text/plain]
Saving to: ‘gpt_generate.py’

gpt_generate.py     100%[===================>]  10.89K  --.-KB/s    in 0s      

2025-11-15 07:24:27 (23.9 MB/s) - ‘gpt_generate.py’ saved [11152/1

In [11]:
from gpt_generate import generate, text_to_token_ids, token_ids_to_text
from previous_chapters import generate_text_simple

In [12]:
# torch.argmax를 사용하는 결정론적 함수

start_context = "Every effort moves you"

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=25,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("출력 텍스트:\n", token_ids_to_text(token_ids, tokenizer))

출력 텍스트:
 Every effort moves you?"

"Yes--quite insensible to the irony. She wanted him vindicated--and by me!"




In [14]:
# 결정론적 동작: top_k 없음, 온도 스케일링 없음

token_ids = generate(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=25,
    context_size=GPT_CONFIG_124M["context_length"],
    top_k=None,
    temperature=0.0,
)

print("출력 텍스트:\n", token_ids_to_text(token_ids, tokenizer))

출력 텍스트:
 Every effort moves you?"

"Yes--quite insensible to the irony. She wanted him vindicated--and by me!"




# 연습문제 5.4: 사전 훈련 계속 수행

In [15]:
!gdown /content/model_and_optimizer.pth

Downloading...
From (original): https://drive.google.com/uc?id=1D_q35z6cqjJTuB3RlD8X7MFmZPF5NI1q
From (redirected): https://drive.google.com/uc?id=1D_q35z6cqjJTuB3RlD8X7MFmZPF5NI1q&confirm=t&uuid=0a1ccf1d-5e35-4244-aba3-63ec73b21af9
To: /content/model_and_optimizer.pth
100% 1.95G/1.95G [00:30<00:00, 63.7MB/s]


In [25]:
import tiktoken
import torch
from previous_chapters import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257, # 어휘 사전 크기
    "context_length": 256, # 짧은 문맥 길이 (원본 길이: 1024)
    "emb_dim": 768, # 임베딩 차원
    "n_heads": 12, # 어텐션 헤드 개수
    "n_layers": 12, # 층 개수
    "drop_rate": 0.1, # 드롭아웃 비율
    "qkv_bias": False, # 쿼리-키-값 생성시 편향 사용 여부
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = tiktoken.get_encoding("gpt2")

checkpoint = torch.load("model_and_optimizer.pth", weights_only=True)

model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(checkpoint["model_state_dict"])

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.1)
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
model.train()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features

In [29]:
import os
import urllib.request
from previous_chapters import create_dataloader_v1

file_path = "the-verdict.txt"
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"

if not os.path.exists(file_path):
  with urllib.request.urlopen(url) as response:
    text_data = response.read().decode('utf-8')
  with open(file_path, "w", encoding="utf-8") as file:
    file.write(text_data)
else:
  with open(file_path, "r", encoding="utf-8") as file:
    text_data = file.read()

# 훈련/검증 비율
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0,
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0,
)

In [24]:
!wget https://bit.ly/4mODn07 -O gpt_train.py

--2025-11-15 07:51:11--  https://bit.ly/4mODn07
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/rickiepark/llm-from-scratch/refs/heads/main/ch05/01_main-chapter-code/gpt_train.py [following]
--2025-11-15 07:51:11--  https://raw.githubusercontent.com/rickiepark/llm-from-scratch/refs/heads/main/ch05/01_main-chapter-code/gpt_train.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8362 (8.2K) [text/plain]
Saving to: ‘gpt_train.py’

gpt_train.py        100%[===================>]   8.17K  --.-KB/s    in 0.001s  

2025-11-15 07:51:11 (14.0 MB/s) - ‘gpt_train.py’ saved [8362/8362]



In [30]:
from gpt_train import train_model_simple

num_epochs = 1
train_losses, val_losses, toekns_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device, num_epochs=num_epochs,
    eval_freq=5, eval_iter=5, start_context="Every effort moves you", tokenizer=tokenizer
)

Ep 1 (Step 000000): Train loss 0.271, Val loss 6.545
Ep 1 (Step 000005): Train loss 0.213, Val loss 6.623
Every effort moves you?"  "Yes--quite insensible to the irony. She wanted him vindicated--and by me!"  He laughed again, and threw back his head to look up at the sketch of the donkey. "There were days when I


# 연습문제 5.5: 사전 훈련된 모델의 훈련 및 검증 세트 손실

In [31]:
import tiktoken
import torch
from previous_chapters import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257, # 어휘 사전 크기
    "context_length": 256, # 짧은 문맥 길이 (원본 길이: 1024)
    "emb_dim": 768, # 임베딩 차원
    "n_heads": 12, # 어텐션 헤드 개수
    "n_layers": 12, # 층 개수
    "drop_rate": 0.1, # 드롭아웃 비율
    "qkv_bias": False, # 쿼리-키-값 생성시 편향 사용 여부
}

torch.manual_seed(123)

tokenizer = tiktoken.get_encoding("gpt2")

In [32]:
!wget https://bit.ly/4kSEn1v -O gpt_download.py

--2025-11-15 08:09:25--  https://bit.ly/4kSEn1v
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/rickiepark/llm-from-scratch/refs/heads/main/ch05/01_main-chapter-code/gpt_download.py [following]
--2025-11-15 08:09:25--  https://raw.githubusercontent.com/rickiepark/llm-from-scratch/refs/heads/main/ch05/01_main-chapter-code/gpt_download.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5972 (5.8K) [text/plain]
Saving to: ‘gpt_download.py’

gpt_download.py     100%[===================>]   5.83K  --.-KB/s    in 0s      

2025-11-15 08:09:25 (41.4 MB/s) - ‘gpt_download.py’ saved [5972/59

In [33]:
from gpt_download import download_and_load_gpt2

settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

checkpoint: 100%|██████████| 77.0/77.0 [00:00<00:00, 125kiB/s]
encoder.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 4.39MiB/s]
hparams.json: 100%|██████████| 90.0/90.0 [00:00<00:00, 58.7kiB/s]
model.ckpt.data-00000-of-00001: 100%|██████████| 498M/498M [00:19<00:00, 25.4MiB/s]
model.ckpt.index: 100%|██████████| 5.21k/5.21k [00:00<00:00, 6.35MiB/s]
model.ckpt.meta: 100%|██████████| 471k/471k [00:00<00:00, 2.70MiB/s]
vocab.bpe: 100%|██████████| 456k/456k [00:00<00:00, 3.21MiB/s]


In [34]:
# 딕셔너리로 모델 설정을 저장
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (124M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (124M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (124M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# 기본 설정을 특정 값으로 업데이트
model_name = "gpt2-small (124M)"  # 모델 이름
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length":1024, "qkv_bias": True})

gpt = GPTModel(NEW_CONFIG)
gpt.eval();

In [35]:
from gpt_generate import load_weights_into_gpt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_weights_into_gpt(gpt, params)
gpt.to(device)

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=7

In [36]:
import os
import urllib.request
from previous_chapters import create_dataloader_v1

file_path = "the-verdict.txt"
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"

if not os.path.exists(file_path):
  with urllib.request.urlopen(url) as response:
    text_data = response.read().decode('utf-8')
  with open(file_path, "w", encoding="utf-8") as file:
    file.write(text_data)
else:
  with open(file_path, "r", encoding="utf-8") as file:
    text_data = file.read()

# 훈련/검증 비율
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0,
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0,
)

In [37]:
from gpt_train import calc_loss_loader

torch.manual_seed(123) # 데이터 로더에서 셔플링을 하므로 재현성을 위해
train_loss = calc_loss_loader(train_loader, gpt, device)
val_loss = calc_loss_loader(val_loader, gpt, device)

print("훈련 손실:", train_loss)
print("검증 손실:", val_loss)

훈련 손실: 3.754749059677124
검증 손실: 3.5596206188201904


In [ ]:
settings, params = download_and_load_gpt2(model_size="1558M", models_dir="gpt2")

model_name = "gpt2-xl (1558M)"
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length":1024, "qkv_bias": True})

gpt = GPTModel(NEW_CONFIG)
gpt.eval();

load_weights_into_gpt(gpt, params)
gpt.to(device)

torch.manual_seed(123)
train_loss = calc_loss_loader(train_loader, gpt, device)
val_loss = calc_loss_loader(val_loader, gpt, device)

print("훈련 손실:", train_loss)
print("검증 손실:", val_loss)

# Colab 무료 버전 작동안함

# 연습문제 5.6: 더 큰 모델 시도하기

settings, params = download_and_load_g
pt2(model_size="124M", models_dir="gpt2")
model_name = "gpt2-small (124M)"

settings, params = download_and_load_gpt2(model_size="1558M", models_dir="gpt2")
model_name = "gpt2-xl (1558M)"